In [1]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime as dt
from datetime import timedelta
import os
import json
import csv
from time import sleep

In [2]:
now_bangkok = (dt.now() + timedelta(hours=7)).isoformat()[:19]
ytd_bangkok = (dt.now() + timedelta(hours=7, days=-30)).isoformat()[:19]

now_YYYY = now_bangkok[:4]
now_mm   = now_bangkok[5:7]
now_dd   = now_bangkok[8:10]

ytd_YYYY = ytd_bangkok[:4]
ytd_mm   = ytd_bangkok[5:7]
ytd_dd   = ytd_bangkok[8:10]

In [3]:
stock = 'ADVANC'
url   = f'https://www.kaohooninternational.com/?s={stock}'

In [4]:
session = requests.Session()
res = session.get(url)
res_text = res.text
soup = BeautifulSoup(res_text)
soup

<!DOCTYPE html>

<html lang="en-US" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#">
<head>
<meta charset="utf-8"/>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<link href="https://www.kaohooninternational.com/xmlrpc.php" rel="pingback"/>
<meta content="width=device-width, initial-scale=1" name="viewport">
<style type="text/css">
					.heateor_sss_button_instagram span.heateor_sss_svg,a.heateor_sss_instagram span.heateor_sss_svg{background:radial-gradient(circle at 30% 107%,#fdf497 0,#fdf497 5%,#fd5949 45%,#d6249f 60%,#285aeb 90%)}
											.heateor_sss_horizontal_sharing .heateor_sss_svg,.heateor_sss_standard_follow_icons_container .heateor_sss_svg{
							color: #fff;
						border-width: 0px;
			border-style: solid;
			border-color: transparent;
		}
					.heateor_sss_horizontal_sharing .heateorSssTCBackground{
				color:#666;
			}
					.heateor_sss_horizontal_sharing span.heateor_sss_svg:hover,.heateor_sss_standard_follow_icons_container span.heateor_sss_svg:hover{


In [5]:
original_reqs = [
    {
        "title": a.select('.entry-title')[0].select('a')[0].string, 
        "href": a.select('.entry-title')[0].select('a')[0].get('href'),
        "text": a.select('.entry-content')[0].string or a.select('.entry-content')[0].text,
        "time": [{
            "datetime": b.get('datetime'),
            "text":     b.text,
        } for b in a.select('time')],
    }
    for a in soup.select('.search-item')
    if len(a.select('.entry-content')[0].text) > 1
]
print(len(original_reqs))
original_reqs

6


[{'title': 'ADVANC Eyes 5% Growth in 2025 from Rising Demand in Mobile and Internet Users',
  'href': 'https://www.kaohooninternational.com/markets/557762',
  'text': '\nSomruetai Tantakitti, Head of Investor Relations of Advanced Info Service Public Company Limited (SET: ADVANC), revealed that the company is still facing macroeconomic risk from the end of the first quarter and early second quarter. ADVANC maintains its EBITDA’s goal this year of 3 – 5%, despite its rate in 1Q25 having increased by 7.4%. […]\n',
  'time': [{'datetime': '2025-05-15T17:22:27+07:00', 'text': '15/05/2025'}]},
 {'title': 'B.Grimm Power Announces 51.6% Increase in 1Q25 Profit, Advancing IUs Expansion and Clean Energy Projects',
  'href': 'https://www.kaohooninternational.com/markets/557711',
  'text': '\nDr. Harald Link, Group President of B.Grimm Power Public Company Limited (SET: BGRIM), revealed that the financial performance in the first quarter of 2025 grew strongly, with Normalized Net Profit (NNP) att

In [6]:
daily_dir_path_1 = f'../data/1_raw_response/04_kaohoon/{now_YYYY}-{now_mm}-{now_dd}'
daily_dir_path_2 = f'../data/2_clean_response/04_kaohoon/{now_YYYY}-{now_mm}-{now_dd}'
daily_dir_path_3 = f'../data/3_raw_news/04_kaohoon/{now_YYYY}-{now_mm}-{now_dd}'

if not os.path.isdir(daily_dir_path_1):
    os.makedirs(daily_dir_path_1)
    
if not os.path.isdir(daily_dir_path_2):
    os.makedirs(daily_dir_path_2)
    
if not os.path.isdir(daily_dir_path_3):
    os.makedirs(daily_dir_path_3)

In [7]:
for i_news, original_req in enumerate(original_reqs):
    try:
        res = requests.get(original_req['href'])
        print(f'Success {i_news:05d}.')
    except:
        print(f'Fail to fetch url `{original_req['href']}`.')

    with open(f'{daily_dir_path_1}/{i_news:05d}.json', 'w') as json_file:
        json.dump(
            {
                "original_req": original_req,
                "status_code": res.status_code,
                "text": res.text,
            }, 
            json_file
        )

    sleep(0.2)

Success 00000.
Success 00001.
Success 00002.
Success 00003.
Success 00004.
Success 00005.


# Scrape news title, news content

In [8]:
raw_news_data = []
raw_news_fields = ['news_datetime', 'news_title', 'news_content']

for i_news, original_req in enumerate(original_reqs):
    with open(f'{daily_dir_path_1}/{i_news:05d}.json', 'r') as json_file:
        try:
            res = json.load(json_file)
            res_text = res['text']

            # print(res_text)

            soup = BeautifulSoup(res_text)

            news_datetime = res['original_req']['time'][0]['datetime']
            news_title    = res['original_req']['title']
            news_content  = soup.select('.entry-content')[0].text

            a_news = {
                "news_datetime": news_datetime,
                "news_title":    news_title,
                "news_content":  news_content,
            }

            raw_news_data.append(a_news)

            with open(f'{daily_dir_path_2}/{i_news:05d}.json', 'w') as json_file:
                json.dump(a_news, json_file)
            
            print(f'Success {i_news:05d}. {news_title}')
        except:
            print(f'Fail reading raw json `{a_news['url']}`.')

with open(f'{daily_dir_path_3}/{i_news:05d}.csv', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=raw_news_fields)
    writer.writeheader()
    writer.writerows(raw_news_data)

Success 00000. ADVANC Eyes 5% Growth in 2025 from Rising Demand in Mobile and Internet Users
Success 00001. B.Grimm Power Announces 51.6% Increase in 1Q25 Profit, Advancing IUs Expansion and Clean Energy Projects
Success 00002. Thai Union Secures $150 Million Blue Loan from ADB to Advance Sustainable Shrimp and Climate Goals
Success 00003. UBS Upgrades ADVANC’s Target Price to THB 339 on Strong Growth Prospects
Success 00004. ADVANC Rises 4% after Robust 1Q25 Earnings Growth and Upbeat Year-End Guideline
Success 00005. ADVANC Reports 25% Profit Growth in 1Q25 amid Robust Mobile and Broadband Businesses
